## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-09-02-29-14 +0000,nyt,Live Updates: Senate Advances Measure to Curb ...,https://www.nytimes.com/live/2026/01/08/world/...
1,2026-01-09-02-27-00 +0000,wsj,USD/SGD’s Positive Divergence Raises Prospect ...,https://www.wsj.com/opinion/asian-currencies-c...
2,2026-01-09-02-25-47 +0000,nyt,Federal Agents Shoot 2 During Traffic Stop in ...,https://www.nytimes.com/2026/01/08/us/politics...
3,2026-01-09-02-23-00 +0000,wsj,LG Electronics Expects First Quarterly Operati...,https://www.wsj.com/business/earnings/lg-elect...
4,2026-01-09-02-21-59 +0000,startribune,Twins reach agreements with all arbitration el...,https://www.startribune.com/minnesota-twins-ml...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2352/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
6,trump,90
56,ice,37
9,venezuela,35
155,shooting,28
106,new,24


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
59,2026-01-09-00-09-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...,197
129,2026-01-08-22-07-42 +0000,nyt,We Pressed Trump on His Conclusion About the I...,https://www.nytimes.com/2026/01/08/us/politics...,190
244,2026-01-08-17-23-16 +0000,nyt,"After ICE Shooting, Trump Administration Deplo...",https://www.nytimes.com/2026/01/08/us/politics...,183
116,2026-01-08-22-40-00 +0000,wsj,The architect of a new White House ballroom to...,https://www.wsj.com/politics/policy/white-hous...,177
74,2026-01-08-23-47-49 +0000,nyt,"Trump Addresses Venezuela, Greenland and Presi...",https://www.nytimes.com/2026/01/08/us/politics...,174


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
59,197,2026-01-09-00-09-00 +0000,wsj,President Trump lashed out at five GOP senator...,https://www.wsj.com/politics/policy/senate-vot...
393,117,2026-01-08-06-05-45 +0000,latimes,'Public execution.' 'Trigger-happy.' Californi...,https://www.latimes.com/california/story/2026-...
116,87,2026-01-08-22-40-00 +0000,wsj,The architect of a new White House ballroom to...,https://www.wsj.com/politics/policy/white-hous...
196,47,2026-01-08-19-14-24 +0000,nypost,Protests erupt in Iranian capital after exiled...,https://nypost.com/2026/01/08/world-news/prote...
110,44,2026-01-08-22-46-01 +0000,nyt,Bondi Warns Minnesota Protesters to Stay Away ...,https://www.nytimes.com/2026/01/08/us/politics...
76,43,2026-01-08-23-42-00 +0000,wsj,A middle-of-the-night deal back in early July ...,https://www.wsj.com/politics/policy/aca-vote-h...
410,42,2026-01-08-03-28-00 +0000,wsj,Trump Team Works Up Sweeping Plan to Control V...,https://www.wsj.com/business/energy-oil/trump-...
286,39,2026-01-08-14-56-31 +0000,nypost,‘CBS Evening News’ No. 2 producer fired during...,https://nypost.com/2026/01/08/media/cbs-evenin...
274,36,2026-01-08-15-32-14 +0000,nypost,"JD Vance predicts ‘dumbest’ of Gavin Newsom, K...",https://nypost.com/2026/01/08/us-news/jd-vance...
78,35,2026-01-08-23-40-30 +0000,nypost,Democrats dig-in on anti-ICE rhetoric as prote...,https://nypost.com/2026/01/08/us-news/democrat...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
